<a href="https://colab.research.google.com/github/profsuccodifrutta/patch_core_brain_mri/blob/main/patch_core_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
zip_path = '/content/drive/MyDrive/brainmri.zip'
extract_path = '/content/brain_dataset' # Cartella locale di Colab

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Estrazione completata!")

✅ Estrazione completata!


In [6]:
!pip install anomalib
!pip install faiss-cpu
!pip freeze > requirements.txt

In [4]:
import torch
from pathlib import Path
from anomalib.data import Folder
from anomalib.models import Patchcore
from anomalib.engine import Engine


DATA_PATH = Path('/content/brain_dataset')

datamodule = Folder(
    name="brain_mri",
    root=DATA_PATH,
    normal_dir="Training/notumor",      # Sani per il training
    normal_test_dir="Testing/notumor",  # Sani per il test
    abnormal_dir="Testing",             # Tutte le altre cartelle in Testing sono anomalie
    train_batch_size=8,
    eval_batch_size=8,
    num_workers=2
)
datamodule.setup()

print("✅ Configurazione completata. Pronto per l'estrazione delle feature.")

✅ Configurazione completata. Pronto per l'estrazione delle feature.


In [ ]:
# PatchCore
model = Patchcore(
    backbone="wide_resnet50_2",
    layers=["layer2", "layer3"],
    coreset_sampling_ratio=0.01
)

# motore colab
engine = Engine(
    max_epochs=1,
    devices=1,
    accelerator="auto"
)

# Crea la memoria dei sani (Training)
print("Creazione Memory Bank dei sani...")
engine.fit(model=model, datamodule=datamodule)

#  Confronta i malati con la memoria (Inference)
print(" Valutazione sulle anomalie...")
results = engine.test(model=model, datamodule=datamodule)

print("\n--- RISULTATI FINALI ---")
print(results)

In [ ]:
# salva il modello
path_salvataggio = "/content/drive/MyDrive/patchcore_model.ckpt"
engine.save_checkpoint(path_salvataggio)
print(f"✅ Modello salvato correttamente in: {path_salvataggio}")

In [ ]:
# inferenza senza dover ricreare la memory bank
# Inizializzi il modello
model = Patchcore(backbone="wide_resnet50_2", layers=["layer2", "layer3"])

# Carica pesi e la Memory Bank dal file
model = Patchcore.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/patchcore_model.ckpt")

# Inizializzi l'Engine solo per il test/predict
engine = Engine(devices=1)

# 4. Fai inferenza su una nuova cartella di immagini
# (Oppure su una singola immagine usando engine.predict)
predictions = engine.predict(model=model, datamodule=datamodule)

In [ ]:
# Visualizzazione Risultati
from anomalib.utils.post_processing import visualizer
import matplotlib.pyplot as plt
from PIL import Image


predictions = engine.predict(model=model, datamodule=datamodule)

# Visualizza le prime 3 immagini del test set
for i, prediction in enumerate(predictions):
    if i > 2: break

    plt.figure(figsize=(12, 4))

    # Immagine Originale
    plt.subplot(1, 3, 1)
    plt.imshow(prediction.image)
    plt.title("Originale")

    # Mappa di Anomalia (Heatmap)
    plt.subplot(1, 3, 2)
    plt.imshow(prediction.anomaly_map, cmap='jet')
    plt.title("Mappa Anomalia")

    # Sovrapposizione (Overlay)
    plt.subplot(1, 3, 3)
    plt.imshow(prediction.segmentations)
    plt.title("Risultato Finale")

    plt.show()